In [ ]:
#%%bash
#export TF_MIN_GPU_MULTIPROCESSOR_COUNT=4
#TF_MIN_GPU_MULTIPROCESSOR_COUNT=4
#export CUDA_VISIBLE_DEVICES=0,1
#CUDA_VISIBLE_DEVICES=0,1

In [1]:
import tensorflow as tf
import numpy as np
import glob
import cv2
from contextlib import contextmanager
import os
import sys



# Helper Functions

In [3]:
def standarize(train_set):
    mean = np.mean(train_set, axis=(0,1,2), keepdims=True)
    std = np.std(train_set, axis=(0,1,2), keepdims=True)
    return (train_set - mean) / std

def scale_range(input, min=-1, max=1):

    input += -(np.min(input))

    input /= np.max(input) / (max - min)

    input += min

    return input

def normalize(train_set, axis=(1,2)):
    maxx = np.max(train_set, axis=axis, keepdims=True)
    minn = np.min(train_set, axis=axis, keepdims=True)
    return (train_set - minn) / (maxx - minn)
    


def load_image(addr):
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.imread(addr)
    img = cv2.resize(img, (320, 240))
    img = normalize(img, axis=(0,1))
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img




In [3]:

def back_conv_layer(x, target_shape, kernel_size, stride_size, name, normalize = False, activation_function = False):
        
    with tf.variable_scope(name):
        print("Back-Conv-Layer:" + str(x.shape))
        fan_in = int(x.shape[1] * x.shape[2])
        depth = x.shape[-1]

        if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
            var_init = tf.random_normal_initializer(stddev = 2/fan_in)
        else:
            var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        kernels = tf.get_variable("kernels", [kernel_size, kernel_size, target_shape[-1], depth], tf.float32, var_init)

        var_init = tf.constant_initializer(0.0)
        biases = tf.get_variable("biases", [target_shape[-1]], initializer = var_init)

        activation = tf.nn.conv2d_transpose(x, kernels, target_shape, strides = [1, stride_size, stride_size, 1]) + biases

        if normalize:
            activation = batch_norm(activation, [0, 1, 2])

        return activation_function(activation) if callable(activation_function) else activation


def _pop_batch_norm(x, pop_mean, pop_var, offset, scale):
    return tf.nn.batch_normalization(x, pop_mean, pop_var, offset, scale, 1e-6)

def _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale):
    decay = 0.99
    
    dependency_1 = tf.assign(pop_mean, pop_mean * decay + mean * (1 - decay))
    dependency_2 = tf.assign(pop_var, pop_var * decay + var * (1 - decay))

    with tf.control_dependencies([dependency_1, dependency_2]):
        return tf.nn.batch_normalization(x, mean, var, offset, scale, 1e-6)

    

def batch_norm(x, axes):
    depth = x.shape[-1]
    mean, var = tf.nn.moments(x, axes = axes)
    
    var_init = tf.constant_initializer(0.0)
    offset = tf.get_variable("offset", [depth], tf.float32, var_init)
    var_init = tf.constant_initializer(1.0)
    scale = tf.get_variable("scale", [depth], tf.float32, var_init)
    
    pop_mean = tf.get_variable("pop_mean", [depth], initializer = tf.zeros_initializer(), trainable = False)
    pop_var = tf.get_variable("pop_var", [depth], initializer = tf.ones_initializer(), trainable = False)
    
    return tf.cond(
        is_training,
        lambda: _batch_norm(x, pop_mean, pop_var, mean, var, offset, scale),
        lambda: _pop_batch_norm(x, pop_mean, pop_var, offset, scale)
    )

def conv_layer(x, kernel_quantity, kernel_size, stride_size, name, normalize = False, activation_function = False):
    #print("Conv-Layer:" + str(x.shape))
    
    with tf.variable_scope(name):
        depth = x.shape[-1]
        fan_in = int(x.shape[1] * x.shape[2])

        if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
            var_init = tf.random_normal_initializer(stddev = 2/fan_in)
        else:
            var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        kernels = tf.get_variable("kernels", [kernel_size, kernel_size, depth, kernel_quantity], tf.float32, var_init)

        var_init = tf.constant_initializer(0.0)
        biases = tf.get_variable("biases", [kernel_quantity], initializer = var_init)

        activation = tf.nn.conv2d(x, kernels, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
        
        if normalize:
            activation = batch_norm(activation, [0, 1, 2])
        
        return activation_function(activation) if callable(activation_function) else activation
    
def flatten(x):
    size = int(np.prod(x.shape[1:]))
    return tf.reshape(x, [-1, size])


def feed_forward_layer(x, target_size, name, normalize = False, activation_function = None):
    with tf.variable_scope(name):
    
        fan_in = int(x.shape[-1])

        if activation_function == tf.nn.relu or activation_function == tf.nn.leaky_relu:
            var_init = tf.random_normal_initializer(stddev = 2/fan_in)
        else:
            var_init = tf.random_normal_initializer(stddev = fan_in**(-1/2))
        weights = tf.get_variable("weights", [x.shape[1], target_size], tf.float32, var_init)

        var_init = tf.constant_initializer(0.0)
        biases = tf.get_variable("biases", [target_size], tf.float32, var_init)

        activation = tf.matmul(x, weights) + biases

        if normalize:
            activation = batch_norm(activation, [0])

        return activation_function(activation) if callable(activation_function) else activation

    
def batch_norm_2(x, is_training):
    return tf.layers.batch_normalization(x, training=is_training)



def norm(tensor):
    return tf.div(
        tf.subtract(tensor, tf.reduce_min(tensor)), 
        tf.subtract(tf.reduce_max(tensor), tf.reduce_min(tensor))
)


PS_OPS = ['Variable', 'VariableV2', 'AutoReloadVariable']

def assign_to_device(device, ps_device='/cpu:0'):
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return "/" + ps_device
        else:
            return device
    return _assign 


# Generator 

In [4]:
   
def generator(images, image_size, reuse=False):
        
    with tf.variable_scope('gen', reuse=reuse):

        half_sized_images = tf.image.resize_images(images, [image_size[0], image_size[1]])
        base_model = tf.keras.applications.DenseNet169(input_shape=(None, None, 3),
                                                       include_top=False, 
                                                       weights='imagenet')
        pre_model = tf.keras.models.Model(inputs=base_model.input, outputs=base_model.get_layer('pool4_relu').output)
        path_1 = pre_model(images)
        path_2 = pre_model(half_sized_images)

        #path_1 = tf.image.resize_bilinear(
        #    path_1,[12, 20],
        #)
        path_2 = tf.image.resize_bilinear(
            path_2,[15, 20],
        )
        encoder_output = tf.concat([path_1, path_2], 3)

        decoder = conv_layer(encoder_output, 1024, 3, 1, "conv_1_1", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 1024, 3, 1, "conv_1_2", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 1024, 3, 1, "conv_1_3", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = back_conv_layer(decoder, [BATCH_SIZE, 30, 40, 512], 3, 2, "deconv_1")

        decoder = conv_layer(decoder, 512, 3, 1, "conv_2_1", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 512, 3, 1, "conv_2_2", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 512, 3, 1, "conv_2_3", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = back_conv_layer(decoder, [BATCH_SIZE, 60, 80, 256], 3, 2, "deconv_2")

        decoder = conv_layer(decoder, 256, 3, 1, "conv_3_1", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 256, 3, 1, "conv_3_2", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 256, 3, 1, "conv_3_3", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = back_conv_layer(decoder, [BATCH_SIZE, 120, 160, 128], 3, 2, "deconv_3")

        decoder = conv_layer(decoder, 128, 3, 1, "conv_4_1", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 128, 3, 1, "conv_4_2", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = back_conv_layer(decoder, [BATCH_SIZE, 240, 320, 64], 3, 2, "deconv_4")

        decoder = conv_layer(decoder, 64, 3, 1, "conv_5_1", normalize=True, activation_function = tf.nn.leaky_relu)
        decoder = conv_layer(decoder, 64, 3, 1, "conv_5_2", normalize=True, activation_function = tf.nn.leaky_relu)

        output = conv_layer(decoder, 1, 1, 1, "conv1x1", activation_function = tf.sigmoid)

    return output

# Discriminator

In [5]:

def discriminator(x, reuse=False):

    with tf.variable_scope('disc', reuse=reuse):

        # Typical convolutional neural network to classify images.

        x = tf.layers.conv2d(x, 64, 5)
        

        x = tf.nn.leaky_relu(x)

        x = tf.layers.average_pooling2d(x, 2, 2)

        x = tf.layers.conv2d(x, 128, 5)

        x = tf.nn.leaky_relu(x)

        x = tf.layers.average_pooling2d(x, 2, 2)
        
        x = tf.layers.conv2d(x, 256, 5)

        x = tf.nn.leaky_relu(x)

        x = tf.layers.average_pooling2d(x, 2, 2)

        x = tf.contrib.layers.flatten(x)

        x = tf.layers.dense(x, 1024)

        x = tf.nn.leaky_relu(x)

        # Output 2 classes: Real and Fake images

        x = tf.layers.dense(x, 2)

    return x
        
    

### Retrieval of filename lists 

In [2]:


trainset_files = np.array(os.listdir("./salicon_dataset/train/images/"))
trainmaps_files = np.array(os.listdir("./salicon_dataset/train/fixations/"))


valset_files = np.array(os.listdir("./salicon_dataset/val/images/"))
valmaps_files = np.array(os.listdir("./salicon_dataset/val/fixations/"))

# Training

In [ ]:
#for training


if not os.path.exists('models/'):
    os.makedirs('models/')
if not os.path.exists('summaries/'):
    os.makedirs('summaries/')
    
BATCH_SIZE = 5
capacity = 10000
threads = 1
samples_nr = 10000
epochs = 500
pre_epochs = 25
validation_step = 100
num_gpus = 1
steps_per_ep = samples_nr//(BATCH_SIZE*num_gpus)
d_iter = 1

#train_path = 'train.tfrecords'
#val_path = 'val.tfrecords'

with tf.device('/cpu:0'):
    

    g_content_losses = []
    g_losses = []
    d_losses = []
    
    image_inputs = tf.placeholder(tf.float32, shape=(BATCH_SIZE*num_gpus, 240, 320, 3))
    image_size_placeholder = tf.placeholder(tf.int32, shape=(2))
    is_training  = tf.placeholder(tf.bool)

    #mode  = tf.placeholder(tf.string)
    real_maps = tf.placeholder(tf.float32, shape=(BATCH_SIZE*num_gpus, 240, 320, 1))
    
    disc_target = tf.placeholder(tf.int32, shape=[BATCH_SIZE*2])
    
    g_global_step = tf.get_variable('g_global_step', trainable=False, initializer=0)

    d_global_step = tf.get_variable('d_global_step', trainable=False, initializer=0)
    
    reuse_var = False
    for i in range(num_gpus):

        with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):
            
            _image_inputs = image_inputs[i * BATCH_SIZE: (i+1) * BATCH_SIZE]

            _real_maps = real_maps[i * BATCH_SIZE: (i+1) * BATCH_SIZE]
            

            gen_output = generator(_image_inputs, image_size_placeholder, reuse=reuse_var)

            disc_real_input = tf.concat([_image_inputs, _real_maps], 3)
            disc_fake_input = tf.concat([_image_inputs, gen_output], 3)


            d_output_real = discriminator(disc_real_input, reuse=reuse_var)
            d_output_fake = discriminator(disc_fake_input, reuse=True)

            d_out_concat = tf.concat([d_output_real, d_output_fake], axis=0)
            D_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=d_out_concat, 
                                                                                   labels=disc_target))

            G_adv_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=d_output_fake, 
                                                                                   labels=disc_target[:BATCH_SIZE]))
            

            G_content_loss = tf.reduce_mean(tf.keras.backend.binary_crossentropy(_real_maps, gen_output))

            G_loss = G_adv_loss + (0.4*G_content_loss)

            
            
            g_losses.append(G_loss)
            d_losses.append(D_loss)
            g_content_losses.append(G_content_loss)
        
        reuse_var = True
  
    
    G_loss = tf.reduce_mean(tf.convert_to_tensor(g_losses))
    G_content_loss = tf.reduce_mean(tf.convert_to_tensor(g_content_losses))
    D_loss = tf.reduce_mean(tf.convert_to_tensor(d_losses))
    
    all_vars = tf.trainable_variables()
    d_vars_dict = {}
    g_vars_dict = {}
    for var in all_vars:
        if "disc" in var.name:
            d_vars_dict[var.name] = var
        if "gen" in var.name:
            g_vars_dict[var.name] = var
    d_vars = list(d_vars_dict.values())
    g_vars = list(g_vars_dict.values())
            
    D_solver = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(D_loss, var_list=d_vars, 
                                                                     global_step=d_global_step,
                                                                    colocate_gradients_with_ops=True)

    G_pre_solver = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(G_content_loss, var_list=g_vars, 
                                                                         global_step=g_global_step,
                                                                        colocate_gradients_with_ops=True)

    G_solver = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(G_loss, var_list=g_vars, 
                                                                     global_step=g_global_step,
                                                                    colocate_gradients_with_ops=True)
    
           

    saver = tf.train.Saver(max_to_keep=5) 


    tf.summary.scalar('G_loss', G_loss)
    tf.summary.scalar('G_content_loss', G_content_loss)
    tf.summary.scalar('D_loss', D_loss)
    #tf.summary.image("Images", image_inputs[:3], max_outputs=3)
    tf.summary.image("Generated", gen_output[:3], max_outputs=3)
    tf.summary.image("Original_maps", real_maps[:3], max_outputs=3)
    merged = tf.summary.merge_all()



    train_ids = np.array((range(len(trainset_files))))
    val_ids = np.array((range(len(valset_files))))
    
    batch_disc_target = np.concatenate([np.ones([BATCH_SIZE]), np.zeros([BATCH_SIZE])], axis=0)

    '''
    ## create random batches when using tf_records
    def get_batch(mode):
        
        feature = {'{}/image'.format(mode): tf.FixedLenFeature([], tf.string),
                   '{}/fixation'.format(mode): tf.FixedLenFeature([], tf.string)}
        # Create a list of filenames and pass it to a queue
        filename_queue = tf.train.string_input_producer(['{}.tfrecords'.format(mode)], num_epochs=1)
        # Define a reader and read the next record
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(filename_queue)
        # Decode the record read by the reader
        features = tf.parse_single_example(serialized_example, features=feature)
        # Convert the image data from string back to the numbers
        image = tf.decode_raw(features['{}/image'.format(mode)], tf.float32)

        # Cast label data into int32
        fixation = tf.decode_raw(features['{}/fixation'.format(mode)], tf.float32)
        # Reshape image data into the original shape
        image = tf.reshape(image, [240, 320, 3])
        fixation = tf.reshape(fixation, [240, 320, 3])
        # Any preprocessing here ...

        # Creates batches by randomly shuffling tensors
        imgs, fixations = tf.train.shuffle_batch([image, fixation], batch_size=BATCH_SIZE, capacity=15, num_threads=1, min_after_dequeue=10)

        fixations = fixations[:,:,:,:1]
        
        return imgs, fixations
    
    tr_batch, tr_fixations = get_batch('train')
    val_batch, val_fixations = get_batch('val')
    '''
    
    config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
    
    init = tf.global_variables_initializer()
    #init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    
    with tf.Session(config=config) as sess:


        train_writer = tf.summary.FileWriter('summaries/train', sess.graph)
        val_writer = tf.summary.FileWriter('summaries/val', sess.graph)

        sess.run(init)
        
        #coord = tf.train.Coordinator()
        #threads = tf.train.start_queue_runners(coord=coord)

        sess.run(d_global_step.initializer)
        sess.run(g_global_step.initializer)

        for epoch in range(1, pre_epochs+1):

            np.random.shuffle(train_ids)

            for train_step in range(steps_per_ep):
                
                img_batch = trainset_files[train_ids[train_step*(BATCH_SIZE*num_gpus):train_step*(num_gpus*BATCH_SIZE)+(BATCH_SIZE*num_gpus)]]
                map_batch = trainmaps_files[train_ids[train_step*(BATCH_SIZE*num_gpus):train_step*(BATCH_SIZE*num_gpus)+(BATCH_SIZE*num_gpus)]]
                
                images = np.array([cv2.resize(cv2.imread("./salicon_dataset/train/images/" + file).astype(np.float32),
                                              (320, 240))
                                   for file in img_batch])
                maps = np.array([np.expand_dims(cv2.resize(cv2.imread("./salicon_dataset/train/fixations/" + file)[:,:,:1].astype(np.float32), 
                                           (320, 240)), axis=-1)
                                     for file in map_batch])
                
                images = standarize(images)
                maps = normalize(maps)
                

                #images, maps = sess.run([tr_batch, tr_fixations])
                
                
                image_size = [images[0].shape[0]//2, images[0].shape[1]//2]


                _ = sess.run(G_pre_solver, feed_dict={image_inputs:images, is_training:True, 
                                                      disc_target:batch_disc_target, image_size_placeholder:image_size,
                                                    real_maps:maps},)




                if tf.train.global_step(sess, g_global_step) % validation_step == 0:

                    summary = sess.run(merged, feed_dict={image_inputs:images, image_size_placeholder:image_size,
                                                          disc_target:batch_disc_target,
                                                                                real_maps:maps, is_training:False,},)


                    train_writer.add_summary(summary, global_step=tf.train.global_step(sess, g_global_step))

                    saver.save(sess, 'models/model.ckpt', global_step=tf.train.global_step(sess, g_global_step))

                    
                    np.random.shuffle(val_ids)
                    
                    img_batch = valset_files[val_ids[:(BATCH_SIZE*num_gpus)]]
                    map_batch = valmaps_files[val_ids[:(BATCH_SIZE*num_gpus)]]

                    
                    images = np.array([cv2.resize(cv2.imread("./salicon_dataset/val/images/" + file).astype(np.float32),
                                              (320, 240))
                                   for file in img_batch])
                    maps = np.array([np.expand_dims(cv2.resize(cv2.imread("./salicon_dataset/val/fixations/" + file)[:,:,:1].astype(np.float32), 
                                           (320, 240)), axis=-1)
                                     for file in map_batch])

                    images = standarize(images)
                    maps = normalize(maps)
                    
                    
                    #images, maps = sess.run([val_batch, val_fixations])

                    image_size = [images[0].shape[0]//2, images[0].shape[1]//2]


                    summary = sess.run(merged, feed_dict={image_inputs:images, image_size_placeholder:image_size,
                                                          disc_target:batch_disc_target,
                                                                                real_maps:maps, is_training:False,},)

                    val_writer.add_summary(summary, global_step=tf.train.global_step(sess, g_global_step))

                    print('Pre-Epoch: {}, step: {}'.format(epoch, tf.train.global_step(sess, g_global_step)))

                    #print('D loss: {:.4}'. format(D_loss_curr))

                    #print('G_loss: {:.4}'.format(gen_loss))

                    #print('D_loss: {:.4}'.format(disc_loss))

                    print()

  
        for epoch in range(1, epochs+1):

            np.random.shuffle(train_ids)

            for train_step in range(steps_per_ep):
                
                
                img_batch = trainset_files[train_ids[train_step*(BATCH_SIZE*num_gpus):train_step*(num_gpus*BATCH_SIZE)+(BATCH_SIZE*num_gpus)]]
                map_batch = trainmaps_files[train_ids[train_step*(BATCH_SIZE*num_gpus):train_step*(BATCH_SIZE*num_gpus)+(BATCH_SIZE*num_gpus)]]
                
                images = np.array([cv2.resize(cv2.imread("./salicon_dataset/train/images/" + file).astype(np.float32),
                                              (320, 240))
                                   for file in img_batch])
                maps = np.array([np.expand_dims(cv2.resize(cv2.imread("./salicon_dataset/train/fixations/" + file)[:,:,:1].astype(np.float32), 
                                           (320, 240)), axis=-1)
                                     for file in map_batch])
                
                images = standarize(images)
                maps = normalize(maps)
                
                
                #images, maps = sess.run([tr_batch, tr_fixations])
                image_size = [images[0].shape[0]//2, images[0].shape[1]//2]


                _ = sess.run(G_solver, feed_dict={image_inputs:images, is_training:True,disc_target:batch_disc_target,
                                                                                 image_size_placeholder:image_size,
                                                                                real_maps:maps},)
                
                for _ in range(d_iter):
                    _ = sess.run(D_solver, feed_dict={image_inputs:images, is_training:True,
                                                      disc_target:batch_disc_target,image_size_placeholder:image_size,
                                                                                real_maps:maps},)
                    


                if tf.train.global_step(sess, g_global_step) % validation_step == 0:

                    summary = sess.run(merged, feed_dict={image_inputs:images, image_size_placeholder:image_size,
                                                          disc_target:batch_disc_target,
                                                                                real_maps:maps, is_training:False,},)

                    train_writer.add_summary(summary, global_step=tf.train.global_step(sess, g_global_step))

                    saver.save(sess, 'models/model.ckpt', global_step=tf.train.global_step(sess, g_global_step))
                    
                    np.random.shuffle(val_ids)
                    
                    img_batch = valset_files[val_ids[:(BATCH_SIZE*num_gpus)]]
                    map_batch = valmaps_files[val_ids[:(BATCH_SIZE*num_gpus)]]

                    
                    images = np.array([cv2.resize(cv2.imread("./salicon_dataset/val/images/" + file).astype(np.float32),
                                              (320, 240))
                                   for file in img_batch])
                    maps = np.array([np.expand_dims(cv2.resize(cv2.imread("./salicon_dataset/val/fixations/" + file)[:,:,:1].astype(np.float32), 
                                           (320, 240)), axis=-1)
                                     for file in map_batch])

                    images = standarize(images)
                    maps = normalize(maps)
                    
                    
                    #images, maps = sess.run([val_batch, val_fixations])
                    image_size = [images[0].shape[0]//2, images[0].shape[1]//2]


                    summary = sess.run(merged, feed_dict={image_inputs:images, image_size_placeholder:image_size,
                                                          disc_target:batch_disc_target,
                                                                                real_maps:maps, is_training:False,},)

                    val_writer.add_summary(summary, global_step=tf.train.global_step(sess, g_global_step))

                    print('Epoch: {}, step: {}'.format(epoch, tf.train.global_step(sess, g_global_step)))

                    print()
        

        # Stop the threads
        #coord.request_stop()

        # Wait for threads to stop
        #coord.join(threads)
        #sess.close()






## Displaying results with test images

In [ ]:
#for restoring models and inference
BATCH_SIZE = 1
capacity = 10000
threads = 1

validation_step = 50
#ids = len(input_set) #//(BATCH_SIZE)


image_inputs = tf.placeholder(tf.float32, shape=(BATCH_SIZE, 240, 320, 3))
image_size_placeholder = tf.placeholder(tf.int32, shape=(2))
is_training  = tf.placeholder(tf.bool)

real_maps = tf.placeholder(tf.float32, shape=(BATCH_SIZE, 240, 320, 1))

g_global_step = tf.get_variable('g_global_step', trainable=False, initializer=0)



gen_output = generator(image_inputs, image_size_placeholder)



all_vars = tf.global_variables()

saver = tf.train.Saver(var_list=all_vars)


config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)


with tf.Session(config=config) as sess:
        saver.restore(sess,tf.train.latest_checkpoint('./models'))
        

        map_list = []
        
        for i in range(len(direc)):
            image = cv2.imread("./test/images/" + direc[i]).astype(np.float32)
            
            height, width = image.shape[0], image.shape[1]
            image = np.expand_dims(cv2.resize(image, (320, 240)), axis=0)
            image = standarize(image)

            image_size = [image.shape[1]//2, image.shape[2]//2]


            gen_map = sess.run(gen_output, feed_dict={image_inputs:image, is_training:False, 
                                                  image_size_placeholder:image_size,
                                                },)
            gen_map = cv2.resize(gen_map[0], (width, height))

            map_list.append(gen_map.squeeze())



In [ ]:
#for showing results

files = glob.glob("./test/images/*.jpg")
files_maps = glob.glob("./test/fixations/*.jpg")
files.sort()
files_maps.sort()
labels = [file[:-4] for file in files]
map_labels = [file[:-4] for file in files_maps]
import matplotlib.pyplot as plt 
from PIL import Image
for i in range(len(files[:1])):
    fig, ax = plt.subplots(3, 1)
    img = np.array(Image.open(files[i]))
    mapp = Image.open(files_maps[i])
    ax[0].imshow(img)
    ax[1].imshow(mapp, cmap="gray")
    ax[2].imshow(map_list[i], cmap="gray")
    ax[0].title.set_text(labels[i])
    ax[1].title.set_text(map_labels[i])
    ax[2].title.set_text(map_labels[i])
    plt.show()


## Creating TF_records 

In [ ]:
#refer to http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html


def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


train_filename = 'train.tfrecords'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(valset_files)):
    # print how many images are saved every 1000 images
    if not i % 1000:
        
        print('train data: {}/{}'.format(i, len(valset_files)))
        sys.stdout.flush()
    # Load the image
    img = load_image("./salicon_dataset/train/images/"+valset_files[i])
    mapp = load_image("./salicon_dataset/train/fixations/"+valmaps_files[i])
    #label = train_labels[i]
    # Create a feature
    feature = {'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring())),
               'train/fixation': _bytes_feature(tf.compat.as_bytes(mapp.tostring()))}
    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Serialize to string and write on the file
    writer.write(example.SerializeToString())

writer.close()
sys.stdout.flush()

# Initial results on Validation of SALICON dataset

In [7]:
def cc(s_map,gt):

    s_map_norm = (s_map - np.mean(s_map))/np.std(s_map)

    gt_norm = (gt - np.mean(gt))/np.std(gt)

    a = s_map_norm

    b= gt_norm

    r = (a*b).sum() / math.sqrt((a*a).sum() * (b*b).sum());

    return r
import math
gt = list(os.listdir("./salicon_dataset/val/fixations/"))
gt.sort()
pred = list(os.listdir("./salicon_val_pred/"))
pred.sort()
cc_l = []
for i in range(len(pred)):
    imgg = cv2.imread("./salicon_dataset/val/fixations/" + gt[i]).astype(np.float32)[:,:,0]
    imgp = cv2.imread("./salicon_val_pred/" + pred[i]).astype(np.float32)[:,:,0]
    cc_l.append(cc(imgp,imgg))

print(np.mean(cc_l))
    

0.8549757535615404


In [8]:
def kldiv(s_map,gt):

    s_map = s_map/(np.sum(s_map)*1.0)

    gt = gt/(np.sum(gt)*1.0)

    eps = 2.2204e-16

    return np.sum(gt * np.log(eps + gt/(s_map + eps)))

import math
gt = list(os.listdir("./salicon_dataset/val/fixations/"))
gt.sort()
pred = list(os.listdir("./salicon_val_pred/"))
pred.sort()
l = []
for i in range(len(pred)):
    imgg = cv2.imread("./salicon_dataset/val/fixations/" + gt[i]).astype(np.float32)[:,:,0]
    imgp = cv2.imread("./salicon_val_pred/" + pred[i]).astype(np.float32)[:,:,0]
    l.append(kldiv(imgp,imgg))

print(np.mean(l))

0.4015174
